# Government Card Expenses - Senators 2019-2022 [BR]

### Importing libraries

In [2]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

### Reading annual transactions files (from local machine)

Every year has its own CSV file downloaded from 'Dados Abertos - CEAPS' website and stored in the local machine. 

In [3]:
year_19  = pd.read_csv('c:/Users/Acer/Documents/Data Projects/Alura - 7DaysOfCode/raw_data/2019.csv', encoding='latin-', delimiter=';', header=1)
year_20  = pd.read_csv('c:/Users/Acer/Documents/Data Projects/Alura - 7DaysOfCode/raw_data/2020.csv', encoding='latin-', delimiter=';', header=1)
year_21  = pd.read_csv('c:/Users/Acer/Documents/Data Projects/Alura - 7DaysOfCode/raw_data/2021.csv', encoding='latin-', delimiter=';', header=1)
year_22  = pd.read_csv('c:/Users/Acer/Documents/Data Projects/Alura - 7DaysOfCode/raw_data/2022.csv', encoding='latin-', delimiter=';', header=1)

Reading the list of senators with their respective affiliation party and state

In [4]:
senators_list  = pd.read_csv('c:/Users/Acer/Documents/Data Projects/Alura - 7DaysOfCode/raw_data/senators_list.csv', encoding='utf-8', delimiter=';')
senators_list

,SENADOR,PARTIDO,UF
0,Acir Gurgacz,PDT,RO
1,Aécio Neves,PSDB,MG
2,Airton Sandoval,MDB,SP
3,Alessandro Vieira,PSDB,SE
4,Alexandre Silveira,PSD,MG
...,...,...,...
210,Wirlande da Luz,MDB,RR
211,Zé Santana,MDB,PI
212,Zenaide Maia,PSD,RN
213,Zequinha Marinho,PL,PA


Setting senators' name uppercase to match the annual transactions tables

In [4]:
senators_list['SENADOR'] = senators_list['SENADOR'].str.upper()

Checking for senators who have changed party during their mandate

In [6]:
senators_list[senators_list['SENADOR'].duplicated()]

,SENADOR,PARTIDO,UF
11,ANTONIO ANASTASIA,PSD,MG
33,DÁRIO BERGER,PSB,SC
35,DAVI ALCOLUMBRE,UNIÃO,AP
55,ELMANO FÉRRER,PP,PI
63,FERNANDO COLLOR,PTB,AL
77,GUARACY SILVEIRA,PP,TO
107,KÁTIA ABREU,PP,TO
110,LASIER MARTINS,PODEMOS,RS
118,LUIZ CARLOS DO CARMO,PSC,GO
133,MARIA DO CARMO ALVES,PP,SE


In [7]:
senators_list[senators_list['SENADOR']=='ANTONIO ANASTASIA']

,SENADOR,PARTIDO,UF
10,ANTONIO ANASTASIA,PSDB,MG
11,ANTONIO ANASTASIA,PSD,MG


### Creating a pandas dataframe for the full period transactions

In [8]:
full_period = year_19.merge(year_20, how='outer').merge(year_21, how='outer').merge(year_22, how='outer')
full_period

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,04/01/2019,Despesa com pagamento de energia elétrica do e...,"66,02",2116543
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,04/01/2019,Despesa com pagamento de energia elétrica do e...,"139,98",2116546
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,07/01/2019,Despesa com pagamento de aluguel de imóvel par...,6000,2113817
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,86161151,25/12/2018,Despesa com pagamento de telefonia para o escr...,"316,39",2116541
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236036,04/02/2019,Despesa com pagamento de energia elétrica para...,"99,45",2116550
...,...,...,...,...,...,...,...,...,...,...,...
69328,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WIXHAI,06/12/2022,"Companhia Aérea: LATAM, Localizador: WIXHAI. P...","2893,04",2191398
69329,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WITOLM,09/12/2022,"Companhia Aérea: GOL, Localizador: WITOLM. Pas...","1180,19",2192272
69330,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,THPKVQ,20/12/2022,"Companhia Aérea: TAM, Localizador: THPKVQ. Pas...","2671,9",2192274
69331,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QNN9HX,21/12/2022,"Companhia Aérea: AZUL, Localizador: QNN9HX. Pa...","1334,31",2192244


### Adding Party and State to the dataframe

In [9]:
full_period = full_period.merge(senators_list, how='left', on='SENADOR')
full_period

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,PARTIDO,UF
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,04/01/2019,Despesa com pagamento de energia elétrica do e...,"66,02",2116543,PDT,RO
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,04/01/2019,Despesa com pagamento de energia elétrica do e...,"139,98",2116546,PDT,RO
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,07/01/2019,Despesa com pagamento de aluguel de imóvel par...,6000,2113817,PDT,RO
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,86161151,25/12/2018,Despesa com pagamento de telefonia para o escr...,"316,39",2116541,PDT,RO
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236036,04/02/2019,Despesa com pagamento de energia elétrica para...,"99,45",2116550,PDT,RO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78678,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WIXHAI,06/12/2022,"Companhia Aérea: LATAM, Localizador: WIXHAI. P...","2893,04",2191398,PL,PA
78679,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WITOLM,09/12/2022,"Companhia Aérea: GOL, Localizador: WITOLM. Pas...","1180,19",2192272,PL,PA
78680,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,THPKVQ,20/12/2022,"Companhia Aérea: TAM, Localizador: THPKVQ. Pas...","2671,9",2192274,PL,PA
78681,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QNN9HX,21/12/2022,"Companhia Aérea: AZUL, Localizador: QNN9HX. Pa...","1334,31",2192244,PL,PA


In [10]:
full_period['VALOR_REEMBOLSADO'] = full_period['VALOR_REEMBOLSADO'].str.replace(',','.')

In [11]:
full_period['VALOR_REEMBOLSADO'] = pd.to_numeric(full_period['VALOR_REEMBOLSADO'])
full_period

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,PARTIDO,UF
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,04/01/2019,Despesa com pagamento de energia elétrica do e...,66.02,2116543,PDT,RO
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,04/01/2019,Despesa com pagamento de energia elétrica do e...,139.98,2116546,PDT,RO
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,07/01/2019,Despesa com pagamento de aluguel de imóvel par...,6000.00,2113817,PDT,RO
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,86161151,25/12/2018,Despesa com pagamento de telefonia para o escr...,316.39,2116541,PDT,RO
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236036,04/02/2019,Despesa com pagamento de energia elétrica para...,99.45,2116550,PDT,RO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78678,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WIXHAI,06/12/2022,"Companhia Aérea: LATAM, Localizador: WIXHAI. P...",2893.04,2191398,PL,PA
78679,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WITOLM,09/12/2022,"Companhia Aérea: GOL, Localizador: WITOLM. Pas...",1180.19,2192272,PL,PA
78680,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,THPKVQ,20/12/2022,"Companhia Aérea: TAM, Localizador: THPKVQ. Pas...",2671.90,2192274,PL,PA
78681,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QNN9HX,21/12/2022,"Companhia Aérea: AZUL, Localizador: QNN9HX. Pa...",1334.31,2192244,PL,PA


### Identifying the companies (suppliers)

The source file presents no data integrity. This can be verified on the 'CNPJ_CPF' and 'FORNECEDOR' columns, which present respectively the supplier's registry code and name. The registry code is an unique value, refering to one and only one supplier. The name, on the other hand,  is freely written by the transaction register, in such way that one supplier can have only one registry code and several different names. Therefore, in order to properly identifying the supplier, it is reasonable to normalize the supplier's name according to its registry code.

In [12]:
company_code = list(full_period['CNPJ_CPF'].unique())
company_code

['05.914.650/0001-66',
 '004.948.028-63',
 '05.423.963/0001-11',
 '13.279.434/0001-14',
 '26.320.603/0001-64',
 '07.087.027/0001-02',
 '08.090.916/0001-91',
 '08.154.750/0001-20',
 '22.848.444/0001-14',
 '08.742.048/0001-87',
 '08.941.827/0001-01',
 '30.637.754/0001-18',
 '10.575.238/0001-07',
 '33.937.681/0001-78',
 '63.764.211/0001-10',
 '06.146.170/0001-65',
 '33.453.598/0054-35',
 '33.453.598/0217-16',
 '84.707.538/0001-20',
 '84.721.745/0001-30',
 '33.856.088/0001-05',
 '22.285.323/0001-01',
 '23.652.846/0001-01',
 '062.135.728-64',
 '13.021.784/0001-86',
 '05.248.037/0001-57',
 '07.446.860/0001-00',
 '18.007.971/0001-29',
 '70.943.345/0001-64',
 '02.059.753/0001-06',
 '19.207.588/0003-49',
 '60.579.703/0001-48',
 '00.896.242/0001-05',
 '04.904.600/0001-35',
 '61.533.949/0001-41',
 '26.894.399/0001-95',
 '25.298.969/0001-11',
 '00.031.708/0001-00',
 '16.978.175/0001-08',
 '28.839.939/0001-63',
 '06.028.059/0001-74',
 '40.432.544/0107-03',
 '26.433.933/0001-66',
 '23.854.919/0001-4

In [13]:
companies_names = []
for i in company_code:
    companies_names.append(full_period['FORNECEDOR'][full_period['CNPJ_CPF']== i].unique()[0])

In [14]:
companies_list=pd.DataFrame( zip(company_code, companies_names), columns = ['CNPJ_CPF', 'nome'])
companies_list

,CNPJ_CPF,nome
0,05.914.650/0001-66,ENERGISA
1,004.948.028-63,GILBERTO PISELO DO NASCIMENTO
2,05.423.963/0001-11,OI MÓVEL S.A.
3,13.279.434/0001-14,E NASCIMENTO AGÊNCIA DE PUBLICIDADE E MARKETIN...
4,26.320.603/0001-64,INFORMANAHORA
...,...,...
7665,04.190.866/0001-63,LUCILENE DOTTO NICOLODI EIRELI
7666,05.044.096/0002-94,POSTO COPA 2002
7667,41.365.622/0001-09,RCM COMUNICAÇÃO E SERVIÇOS LTDA.
7668,23.358.845/0001-59,AM & AP COMÉRCIO DE ALIMENTOS LTDA.-ME


In [15]:
full_period = full_period.merge(companies_list, on='CNPJ_CPF', how='inner')

In [21]:
full_period[['FORNECEDOR','nome']][full_period['FORNECEDOR'] != full_period['nome']]

,FORNECEDOR,nome
23,ENERGISA RONDÔNIA - DISTRIBUIDORA DE ENERGIA S/A,ENERGISA
24,ENERGISA RONDÔNIA - DISTRIBUIDORA DE ENERGIA S/A,ENERGISA
25,ENERGISA RONDÔNIA - DISTRIBUIDORA DE ENERGIA S/A,ENERGISA
26,ENERGISA RONDÔNIA - DISTRIBUIDORA DE ENERGIA S/A,ENERGISA
27,ENERGISA RONDÔNIA - DISTRIBUIDORA DE ENERGIA S/A,ENERGISA
...,...,...
78256,Locauto Rent a Car Locação de Veículos Ltda,LOCAUTO RENT A CAR LOCAÇÃO DE VEÍCULOS LTDA
78423,Posto de combustíveis Cerca Nova,POSTO CERCA NOVA
78427,Vargas Petróleo Ltda,AUTO POSTO PANTANAL
78482,JMI - comércio de derivados de petróleo Ltda,JMI COMERCIO DE DERIVADOS DE PETRÓLEO LTDA.


In [22]:
full_period['FORNECEDOR'] = full_period['nome']
del full_period['nome']
full_period

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,PARTIDO,UF
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,04/01/2019,Despesa com pagamento de energia elétrica do e...,66.02,2116543,PDT,RO
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,04/01/2019,Despesa com pagamento de energia elétrica do e...,139.98,2116546,PDT,RO
2,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236036,04/02/2019,Despesa com pagamento de energia elétrica para...,99.45,2116550,PDT,RO
3,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236031,04/02/2019,Despesa com pagamento de energia elétrica para...,262.93,2116551,PDT,RO
4,2019,3,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007887449,01/03/2019,Pagamento de energia elétrica para o escritóri...,85.50,2116572,PDT,RO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78678,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",04.190.866/0001-63,LUCILENE DOTTO NICOLODI EIRELI,84449,18/06/2022,NaN,400.07,2183890,PL,PA
78679,2022,6,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",05.044.096/0002-94,POSTO COPA 2002,114820,05/06/2022,NaN,525.06,2183892,PL,PA
78680,2022,12,ZEQUINHA MARINHO,Divulgação da atividade parlamentar,41.365.622/0001-09,RCM COMUNICAÇÃO E SERVIÇOS LTDA.,3,01/02/2023,NaN,14260.00,2193283,PL,PA
78681,2022,12,ZEQUINHA MARINHO,"Locomoção, hospedagem, alimentação, combustíve...",23.358.845/0001-59,AM & AP COMÉRCIO DE ALIMENTOS LTDA.-ME,29177,14/12/2022,NaN,43.00,2191576,PL,PA


In [24]:
full_period.dtypes

ANO                    int64
MES                    int64
SENADOR               object
TIPO_DESPESA          object
CNPJ_CPF              object
FORNECEDOR            object
DOCUMENTO             object
DATA                  object
DETALHAMENTO          object
VALOR_REEMBOLSADO    float64
COD_DOCUMENTO          int64
PARTIDO               object
UF                    object
dtype: object

### Connecting to MySQL

In [63]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='12345678',
                             database='senadores',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

### Converting pandas DataFrame to SQL and inserting it into a table on 'Senadores' database, on MySQL

In [64]:
hostname= "127.0.0.1"
database= "senadores"
username= "root"
password= "12345678"

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=database, user=username, pw=password))

In [65]:
full_period.to_sql('denormal', engine, if_exists='replace', index=False)


78683

# DOCUMENTANDO

Uso do Pandas para importar arquivo csv

limpeza básica: tirando cabeçalho, definindo o encoding e o delimitador

uso do pandas para mesclar os arquivos dos anos em uma só tabela desnormalizada

abrindo uma conexão com o banco de dados local do MySQL

Uso do sqlalchemy para exportar o dataframe para o banco de dados do mysql.

No workbench, a tabela desnormalizada foi exportada para o formato csv, de modo a permitir leitura pelo PowerBI

No Power Query do PowerBI foi identificado alguns registros com erros. esses registros foram identificados por um número de 8 dígitos que se encontravam na coluna 'SENADOR', mas pelo tipo percebeu-se que se referia à coluna 'COD_DOCUMENTO', que tem por caracteristica ser um valor único (não duplicado).

No workbench foi possível encontrar exatamente quais os registros que apresentavam este erro. Verificou-se que o erro estava na coluna 'DETALHAMENTO' que, por vezes, apresentava em seu conteúdo os caracteres ',' ou ';' o que gerava confusão no aplicativo, agindo como delimitadores de coluna.

Foi então usada a função UPDATE para corrigir o erro, retirando-se a ',' do texto

Na verdade, ao fazer a consulta no mysql, a tabela foi exportada no formato CSV, mas com o ';' como delimitador, o que acabou com esses erros do Power BI. Outros erros, no entanto ocorreram devido ao fato da descrição da despesa (coluna detalhamento) ter sido muito longa, gerando quebra de linha no editor de textos. Isso fez com que o Power BI interpretasse a linha quebrada como um novo registro, com tipos errados de dados.

dentre os 200 senadores, 15 fizeram troca de partido dentro do periodo avaliado. Por isso, usamos a funcao duplicated() para identificar esses senadores. No MySQL, então, fizemos a substituição do partido de acordo com o ano em ele fez a transacao, embora não seja exato